# PDSG ODOT

Load and explore data.  quickly check for problems.

In [ ]:
So data quality questions.

How has the number of crashes changed?
How has the number of fatalities changed?
Bicycles?

Geographic variation over time?  

Alcohol

Seatbelts effect on injuries vs fatalities.

Change in cyclist, pedestrian injuries.  


More accidents at certain times of day.

Weather


Combinations of impaired driver and certain weather conditions 

do improvements on roadway change accident rate.  

Are there more accidents in residential/interstates?
More fatalities/injuries

Has the number of injuries/fatalities changed over time?

Implementation:
Tools
Visualization  - plot on map - need to convert from corner of... to 


In [140]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
df_clack=pd.read_csv('train_data/Clackamas.csv')
df_mult=pd.read_csv('train_data/Multnomah.csv')
df_wash=pd.read_csv('train_data/Washington.csv')
#df_marion=pd.read_csv('train_data/Marion.csv')

/home/jonathan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,18,20,21,22,33,34,35,36,38,51,58,62,78,106,129,130,132) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


/home/jonathan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,33,58,62,78,106,132) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


/home/jonathan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,78,106,132) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_tot=pd.DataFrame();
df_list=[df_wash,df_clack,df_mult]
for df in df_list:
    df_tot=df_tot.append(df);
df_tot=df_tot.reindex()


In [4]:
for df in df_list:
    del df

## Looking for weird data

There were complaints about some funny, awkwardly formatted columns.

In [94]:
weird_columns=[5,18,20,21,22,33,38,58,62,106,132]

In [91]:
#This plots out the unique values to see how weird each column is.  
for col in weird_columns:
    print(col, df_tot.columns[col],df_tot.iloc[:,col].unique())

5 Vehicle Coded Seq# [nan '1 ' '01' '2 ' '02' '3 ' '03' '  ' '4 ' '04' '5 ' '05' '6 ' '06' '7 ' '07'
 1.0 2.0 3.0 '8 ' '08' 4.0 5.0 6.0 7.0 8.0 '1' '2' '3' '4' '5' '7' '6']
18 Highway Number ['102' nan '047' '   ' '037' '140' '029' '091' '001' '064' '141' '144' '143'
 '040' '142' '002' 1.0 64.0 141.0 144.0 47.0 40.0 143.0 '081' '171' '068' '160'
 '026' '174' '161' '172' '003' '173' '053' '051' '162' '164' '211' '163' '030'
 '072' '1' '47' '64' '26' '81' '123' '68' '61' '92' '91' '2' '3' '100' '120'
 '092' '061']
20 Roadway Number ['1' nan '2' ' ' 1.0 2.0]


21 Highway Component ['0' nan '6' ' ' '1' '3' 0.0 6.0]
22 Mileage Type ['0' nan ' ' 0.0 'Z']
33 Street Number ['               ' nan '01501          ' ..., '00525          '
 '08606          ' '01483          ']


38 Milepoint ['86.84  ' nan '83.02  ' ..., '304.13 ' '41.07  ' '-5.94  ']
58 Investigating Agency [2.0 nan 5.0 1.0 0.0 3.0 4.0 '0' '3' '2' '5' '1' ' ' 7.0 6.0 '7' '4']
62 Crash Level Cause 1 Code 

[1.0 nan 10.0 26.0 16.0 6.0 20.0 11.0 30.0 32.0 7.0 33.0 13.0 27.0 2.0 12.0 5.0
 8.0 3.0 4.0 21.0 22.0 25.0 14.0 15.0 18.0 19.0 24.0 35.0 0.0 31.0 '13' '27'
 '32' '02' '07' '01' '04' '06' '08' '05' '03' '10' '12' '16' '33' '30' '14' '18'
 '26' '  ' '11' 29.0 17.0 40.0 50.0 28.0 '7' '2' '3' '8' '0' '4' '1' '6' '15'
 '5' '20' '24' '31' '00' '25' '22' '19']
78 Segment Point LRS Measure [86.83931432 nan 83.01990929 ..., '410.293006398949    ' '385.779066175346    '
 '96.3714005614723    ']
106 Number of Trailers [nan 0.0 1.0 2.0 '0' '1' ' ' '2' '9' 9.0 '3' 3.0]


132 Participant Safety Equipment Use Code [nan '2' '9' '4' '6' '0' ' ' '3' '1' '5' 9.0 2.0 '8' 4.0 6.0 0.0 1.0]


In [8]:
?np.float32(' ')

# Checking counts

Count total number of crashes, and number of crashes by number of cyclists involved.

In [5]:
# Unique number of crash IDs
df_tot['Crash ID'].unique().shape

(101119,)

In [7]:
df_tot['Total Pedalcyclist Count'].hist(log=True,bins=[0,1,2,3,4])
plt.show()

# Crash heatmaps

So let's visualize where the crashes happen.  Nothing fancy.
Just build up a heatmap based on lat/lon pairs for each crash.  Treats each crash as one value.

In [280]:
df_tot['Crash ID'].unique().shape

(101119,)

In [17]:
from numba import jit
import time

In [18]:
#Need to make a function to convert degrees, minutes,seconds to angle.
dec_degree = lambda x: x[:,0]+np.sign(x[:,0])*(x[:,1]/60.0+x[:,2]/3600.0)

In [19]:
def make_latlon(df):
    lat_list=['Latitude Degrees',
    'Latitude Minutes',
    'Latitude Seconds']

    long_list=['Longitude Degrees',
    'Longitude Minutes',
    'Longitude Seconds']
    lons=dec_degree(df[long_list].values)
    lats=dec_degree(df[lat_list].values)
    msk=(lons==lons)
    lons=lons[msk]
    lats=lats[msk]
    return lats,lons

In [20]:
def make_xydict(lats,longs,Nx=1000):
    """make_xydict
    Make dict to store max/min  for x/y and number of x points. 
    Can then exactly reconstruct grid, and ranges
    """
    #find 99% percentile edges
    xd={}
    xd['xmin']=np.percentile(longs,0.5);
    xd['xmax']=np.percentile(longs,99.5);

    xd['ymin']=np.percentile(lats,0.5);
    xd['ymax']=np.percentile(lats,99.5);
    xd['Nx']=Nx;
    xd['dx']=(xd['xmax']-xd['xmin'])/Nx
    xd['Ny']=int((xd['ymax']-xd['ymin'])/xd['dx'])
    
    return xd


In [94]:
@jit
def make_eff_heatmap(lats,longs,xd,sigma_fac=1):
    """make_eff_heatmap

    Make a heatmap with a given kernel for smearing
    given a list of lats/longitudes.  

    Input: lats - numpy array of decimal latitudes
           longs - numpy array of decimal longitudes
           xd - dict containing max/min values, Nx and dx for X(lon) and Y(lat) dimensions
           sigma_fac - number of bins for blurring
    Output: heatmap - (Nx, Ny) numpy array 
    """
    Nv = len(lats)

    pad = sigma_fac*3;

    Nx_tot=xd['Nx']+2*pad;    
    Ny_tot=xd['Ny']+2*pad;

    #Make the kernel to add up everywhere.
    Np=2*pad+1;
    kernel=np.zeros((2*pad+1,2*pad+1));
    mu=pad+1
    for i in range(Np):
        for j in range(Np):
            kernel[i,j]=np.exp(-((i-mu)**2+(j-mu)**2)/(2*sigma_fac**2));
    
    #make the heatmap
    heat_tot = np.zeros((Nx_tot,Ny_tot));
    for i in range(Nv):
        #find the pixel corresponding to upper edge.
        #Note max/min for x/y.  Assuming (0,0) is upper left.
        xi = int((longs[i]-xd['xmin'])/xd['dx'])+pad;
        yi = int((xd['ymax']-lats[i])/xd['dx'])+pad;
        #check that box falls inside allowed region.
        if ((xi>0) & ((xi+Np)<Nx_tot) & (yi>0) & ((yi+Np)<Ny_tot)):
           heat_tot[xi:xi+Np, yi:yi+Np]+=kernel
    #now remove the padding so max/min are as specified.
    #hopefully counting is not off by one.
    #Take transpose because of way python counts, so y goes along rows, x along columns.
    heat_tot=heat_tot[pad:-pad,pad:-pad].T
    return heat_tot


In [22]:
df_sub=df_tot.iloc[:-1:1000]
lats,lons=make_latlon(df_sub)
xydict=make_xydict(lats,lons)
heat_sub=make_eff_heatmap(lats,lons,xydict)
plt.figure(figsize=(10,6))
plt.plot(lons,lats,'x')
plt.axis([xydict['xmin'],xydict['xmax'],xydict['ymin'],xydict['ymax']])
plt.show()

In [45]:
#Double lambda: converts numpy aray to list of strings to 4d.p.
fmt_4dp= lambda y: list(map(lambda x: '{:.4f}'.format(x),y))

In [47]:
plt.imshow(heat_sub)
Ntag_x=5;
xticks_bin=np.linspace(0,heat_sub.shape[1],5)
xticks_val=np.linspace(xydict['xmin'],xydict['xmax'],5)
yticks_bin=np.linspace(0,heat_sub.shape[0],5)
yticks_val=np.linspace(xydict['ymax'],xydict['ymin'],5)
plt.xticks(xticks_bin,fmt_4dp(xticks_val),rotation='vertical')
plt.yticks(yticks_bin,fmt_4dp(yticks_val))
plt.colorbar()
plt.show()

In [103]:
def make_heatmap_plot(heat,xd,title_str,savename=None):
    plt.figure(figsize=(20,10))
    #shift by one so that zero counts return zero, since log(1)=0.
    plt.imshow(np.log10(heat+1),cmap=plt.cm.hot)
    #Make ticks to 4d.p.
    #Could just choose round numbers, but meh to that. 
    fmt_4dp= lambda y: list(map(lambda x: '{:.4f}'.format(x),y))
    xticks_bin=np.linspace(0,heat.shape[1],5)
    xticks_val=np.linspace(xydict['xmin'],xydict['xmax'],5)
    yticks_bin=np.linspace(0,heat.shape[0],5)
    yticks_val=np.linspace(xydict['ymax'],xydict['ymin'],5)
    plt.xticks(xticks_bin,fmt_4dp(xticks_val),rotation='vertical')
    plt.yticks(yticks_bin,fmt_4dp(yticks_val))
    #Labelling
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(title_str)
    plt.colorbar()
    if savename is not None:
        plt.savefig(savename)
    plt.show()

In [104]:
t0=time.time()
lats,lons=make_latlon(df_tot)
xd=make_xydict(lats,lons,Nx=2000)
heat = make_eff_heatmap(lats,lons,xd,sigma_fac=1)
t1=time.time()
print('time taken',t1-t0)
make_heatmap_plot(heat,xd,'Log10-number of ALL accidents from 2012-2015 in Portland Area','crash_heatmap.pdf')

time taken 3.1792945861816406


In [98]:
cycle_msk=df_tot['Total Pedalcyclist Count']>0
df_cycle=df_tot[cycle_msk]

In [101]:
clats, clons = make_latlon(df_cycle)
cheat=make_eff_heatmap(clats,clons,xd,sigma_fac=1)
make_heatmap_plot(cheat,xd,'Log10 number of BICYCLE accidents from 2012-2015 in Portland Area')

In [88]:
cheat.sum()

0.0

In [ ]:
# Try counting number of crashes per segment marker. 

In [119]:
seg_groups=df_tot.groupby('Segment Marker ID')

In [122]:
seg_counts=seg_groups.count().values[:,0]

In [132]:
plt.figure()
plt.hist(np.log10(seg_counts[1:]),log=True)
plt.xlabel('Number of crashes in a segment')
plt.ylabel('Number of segments')
plt.show()

In [126]:
seg_counts[1:]

array([12, 21, 26, ...,  8,  4,  2])

So first issue: empty spaces.  I'm going to make an assumption that empty space is a missing value.
Looks like the first entry for each Crash ID has general info, and further columns describe each participant.  

In [11]:
#Gets number of crashes, and counts them.
len(df_crash.count())

24535

In [30]:
df_yr=df_tot.groupby(['Crash Year'])

In [34]:
df_yr.apply(sum)

KeyboardInterrupt: 

In [ ]:
df_tot['Crash Year'].hist(bins=20)
plt.show()